In [2]:
import pandas as pd
import geopandas as gpd
import requests, json
%matplotlib inline

AAnn = pd.read_csv('Austin_Annual_Crime_Dataset_2015.csv')
AUF2015= pd.read_csv("Austin_UF_R2R_2015.csv")

#When writing actual script, need to 'within line in open(...) and call directly from internet.

In [3]:
print(stack.keys())

print(AAnnKeys)

print(stack.columns.get_loc('GOPrimaryKey'))
print(stack.columns.get_loc('Council_District'))
print(stack.columns.get_loc('GOHighestOffenseDesc'))
print(stack.columns.get_loc('NIBRS'))
print(stack.columns.get_loc('GOReportDate'))
print(stack.columns.get_loc('GOLocation'))

print(stack.columns.get_loc('ClearanceStatus'))
print(stack.columns.get_loc('ClearanceDate'))
print(stack.columns.get_loc('GODistrict'))
print(stack.columns.get_loc('GOLocationZip'))
print(stack.columns.get_loc('GOCensusTract'))
print(stack.columns.get_loc('GOXCoordinate'))
print(stack.columns.get_loc('GOYCoordinate'))



NameError: name 'stack' is not defined

In [9]:
#Renaming columns
AUF2015.rename(columns={' Primary Key': 'Key', ' Effect on Officer': ' OfficerEffects', 'Nature of Contact':'NatureOfContact', 'Officer Yrs of Service': 'OfficerYrsServ'}, inplace=True)
AAnn.rename(columns={'HighestNIBRS/UCROffenseDescription':'NIBRS', 'CouncilDistrict': 'Council_District'}, inplace=True)

#Removing spaces in column names
AUF2015.columns = AUF2015.columns.str.replace('\s+','')
AAnn.columns = AAnn.columns.str.replace('\s+','')

#List of AAnn keys for referencing 
#AAnnKeys = AAnn.keys().tolist()
#AAnnKeys2= [11, 4, 8, 18, 12, 9, 2, 1, 7, 10, 6, 13, 14]
    
stack = pd.merge(AAnn, AUF2015, left_on='GOPrimaryKey', right_on='Key', how='outer')

#Combining datasets
#stack = pd.merge([AAnn, AUF2015], a)

#Removing hyphens from index names
#stack.rename(columns={'X-Coordinate':'XCoord', 'Y-Coordinate':'YCoord'}, inplace = True)

#Creating Boolean for UF incidents
#stack['UF'] = stack['AreaCommand'].isnull()

#Copying the info from the AAnn report into the row with the AUF2015 report. 
#for r in stack['Key'].isin(dupkeylist):
#    if r == True:
#        for x in AAnnKeys:
#            stack[x].fillna(AAnn[x], inplace = True)

#Merging key and council district info
#stack['Key'].fillna(stack['GOPrimaryKey'], inplace=True)
#stack.drop(['GOPrimaryKey'], axis = 1, inplace = True)

#stack['CouncilDistrict'].fillna(stack['Council_District'], inplace=True)
#stack.drop(['Council_District'], axis = 1, inplace = True)

#stack['XCoord'].fillna(stack['GOXCoordinate'], inplace=True)
#stack.drop(['GOXCoordinate'], axis = 1, inplace = True)

#stack['YCoord'].fillna(stack['GOYCoordinate'], inplace=True)
#stack.drop(['GOYCoordinate'], axis = 1, inplace = True)

#Dropping the crime reports that are present in both AAnn and AUF2015
#stack = stack.drop_duplicates(subset='Key', keep='last', inplace = False)

#Reordering index for ease of navigating dataset
#stack = stack.reindex_axis(['Key','CouncilDistrict','UF','XCoord','YCoord','RIN',
                           # 'DateOccurred','TimeOccurred','ClearanceDate', 'GOReportDate',
                            #'R2RLevel','NIBRS','AreaCommand',  'Location', 'GOCensusTract',
                            #'GODistrict','GOLocation','GOLocationZip','OfficerEffects','OfficerCommissionDate',
                            #'OfficerYrsServ', 'OfficerOrganizationDesc', 'ReasonDesc','SubjectConductDesc', 'SubjectEffects',
                            #'SubjectEthnicity', 'SubjectRace', 'SubjectResistance', 'SubjectSex', 'NatureOfContact',
                            #'GOHighestOffenseDesc', 'NumberShots', 'WeaponUsed1', 'WeaponUsed2', 'WeaponUsed3', 'WeaponUsed4',
                            #'WeaponUsed5', 'ClearanceStatus','MasterSubjectID'], axis=1)


#Sorting by council district
#stack.sort_values(('CouncilDistrict'), inplace = True)

#Resetting index to Key
#stack.set_index(['Key'], drop = False, inplace = True)

#Saving to csv
#stack.to_csv('stack.csv')

#Saving to csv
#stack.to_csv('stack_testC.csv')


In [11]:
stack
#fuck

,GOPrimaryKey,Council_District,GOHighestOffenseDesc,NIBRS,GOReportDate,GOLocation,ClearanceStatus,ClearanceDate,GODistrict,GOLocationZip,...,WeaponUsed5,NumberShots,SubjectEffects,OfficerEffects,OfficerOrganizationDesc,OfficerCommissionDate,OfficerYrsServ,X-Coordinate,Y-Coordinate,CouncilDistrict
0,2.015108e+08,4.0,AGG ROBBERY/DEADLY WEAPON,Robbery,1-Jan-15,9001 N IH 35 SVRD NB ...,N,28-Jan-15,E,78753.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.015112e+08,4.0,ROBBERY BY ASSAULT,Robbery,1-Jan-15,919 E KOENIG LN SVRD EB ...,N,13-Jan-15,I,78751.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2.015117e+08,1.0,BURGLARY OF RESIDENCE,Burglary,1-Jan-15,12151 N IH 35 SVRD NB ...,N,13-Jan-15,E,78753.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2.015114e+08,4.0,BURGLARY OF RESIDENCE,Burglary,1-Jan-15,1044 NORWOOD PARK BLVD ...,N,5-Jan-15,I,78753.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2.015119e+08,2.0,BURGLARY OF RESIDENCE,Burglary,1-Jan-15,2413 BITTER CREEK DR ...,N,7-Jan-15,F,78744.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2.015114e+08,3.0,BURGLARY OF RESIDENCE,Burglary,1-Jan-15,4600 ELMONT DR ...,N,7-Jan-15,H,78741.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2.015114e+08,9.0,BURGLARY OF RESIDENCE,Burglary,1-Jan-15,1500 E RIVERSIDE DR ...,N,25-Feb-15,H,78741.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2.015108e+08,7.0,BURGLARY NON RESIDENCE,Burglary,1-Jan-15,2500 W PARMER LN ...,N,16-Jan-15,A,78727.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2.015106e+08,1.0,BURGLARY OF RESIDENCE,Burglary,1-Jan-15,1400 MARCUS PL ...,N,13-Jan-15,C,78721.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2.015116e+08,2.0,BURGLARY OF RESIDENCE,Burglary,1-Jan-15,4600 NUCKOLS CROSSING RD ...,N,7-Jan-15,F,78744.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
print(AAnn.keys())
print(AUF2015.keys())

Index(['GOPrimaryKey', 'CouncilDistrict', 'GOHighestOffenseDesc',
       'HighestNIBRS/UCROffenseDescription', 'GOReportDate', 'GOLocation',
       'ClearanceStatus', 'ClearanceDate', 'GODistrict', 'GOLocationZip',
       'GOCensusTract', 'GOXCoordinate', 'GOYCoordinate'],
      dtype='object')
Index(['RIN', 'Key', 'DateOccurred', 'TimeOccurred', 'Location', 'AreaCommand',
       'NatureofContact', 'ReasonDesc', 'R2RLevel', 'MasterSubjectID',
       'SubjectSex', 'SubjectRace', 'SubjectEthnicity', 'SubjectConductDesc',
       'SubjectResistance', 'WeaponUsed1', 'WeaponUsed2', 'WeaponUsed3',
       'WeaponUsed4', 'WeaponUsed5', 'NumberShots', 'SubjectEffects',
       'OfficerEffects', 'OfficerOrganizationDesc', 'OfficerCommissionDate',
       'OfficerYrsServ', 'X-Coordinate', 'Y-Coordinate', 'CouncilDistrict'],
      dtype='object')


In [239]:
print(len(stack))
print(len(AUF2015))
print(len(AAnn))
print(len(dupkeylist))

(len(AAnn) + len(AUF2015)) - (192)

41875
3302
38573
192


41683

In [ ]:
#Find the length of the falses in stacks
#Find length of the trues

#Compare these to the lengths of the orginial AAnn and AUF2015

#Find the Falses where there are NaNs in a GO column

In [188]:
stack['Key'].isin(dupkeylist)

0       False
1       False
2       False
3       False
4       False
5       False
6       False
7       False
8       False
9       False
10      False
11      False
12      False
13      False
14      False
15      False
16      False
17      False
18      False
19      False
20      False
21      False
22      False
23      False
24      False
25      False
26      False
27      False
28      False
29      False
        ...  
3272    False
3273    False
3274    False
3275    False
3276    False
3277    False
3278    False
3279    False
3280    False
3281    False
3282    False
3283    False
3284    False
3285    False
3286    False
3287    False
3288    False
3289    False
3290     True
3291    False
3292    False
3293    False
3294    False
3295    False
3296    False
3297    False
3298    False
3299    False
3300    False
3301    False
Name: Key, Length: 41875, dtype: bool

,Key,CouncilDistrict,UF,XCoord,YCoord,RIN,DateOccurred,TimeOccurred,ClearanceDate,GOReportDate,...,NatureOfContact,GOHighestOffenseDesc,NumberShots,WeaponUsed1,WeaponUsed2,WeaponUsed3,WeaponUsed4,WeaponUsed5,ClearanceStatus,MasterSubjectID
3292,2.015364e+10,9.0,False,3115522.0,10070515.0,170264.0,12/30/2015 12:00:00 AM,0245,3-Feb-15,2-Feb-15,...,NaN,THEFT,NaN,WEAPONLESS (PRESSURE POINTS/KICKS/HAND),NaN,NaN,NaN,NaN,N,459357086: 20153640113
3296,2.015365e+10,9.0,False,3115416.0,10070416.0,169873.0,12/31/2015 12:00:00 AM,,12-Feb-15,2-Feb-15,...,NaN,THEFT,NaN,CHEMICAL AGENT - O C SPRAY,NaN,NaN,NaN,NaN,N,211288398: 20153652230
3298,2.015365e+10,1.0,False,3121574.0,10071826.0,170006.0,12/31/2015 12:00:00 AM,1550,16-Mar-15,2-Feb-15,...,NaN,THEFT,NaN,WEAPONLESS (PRESSURE POINTS/KICKS/HAND),NaN,NaN,NaN,NaN,N,458987405: 20153650995
3299,2.015365e+10,3.0,False,3117764.0,10056344.0,170046.0,12/31/2015 12:00:00 AM,2337,11-Mar-15,2-Feb-15,...,NaN,THEFT,NaN,WEAPONLESS (PRESSURE POINTS/KICKS/HAND),NaN,NaN,NaN,NaN,N,459358256: 20153651957
3301,2.015365e+10,8.0,False,3074396.0,10053463.0,170104.0,12/31/2015 12:00:00 AM,1353,28-Apr-15,2-Feb-15,...,NaN,BREACH OF COMPUTER SECURITY,NaN,WEAPONLESS (PRESSURE POINTS/KICKS/HAND),NaN,NaN,NaN,NaN,N,253427997: 20153650763


In [94]:
AAnnKeys = AAnn.keys().tolist()
AAnnKeys

['GO Primary Key',
 'Council District',
 'GO Highest Offense Desc',
 'Highest NIBRS/UCR Offense Description',
 'GO Report Date',
 'GO Location',
 'Clearance Status',
 'Clearance Date',
 'GO District',
 'GO Location Zip',
 'GO Census Tract',
 'GO X Coordinate',
 'GO Y Coordinate']

In [92]:
print(stack.keys())
#print(AAnn.keys())
#24
#stack.iloc[:, 23:42]

Index(['AreaCommand', 'DateOccurred', 'OfficerEffects', 'Location',
       'NatureOfContact', 'NumberShots', 'OfficerCommissionDate',
       'OfficerOrganizationDesc', 'Key', 'R2RLevel', 'RIN', 'ReasonDesc',
       'SubjectConductDesc', 'SubjectEffects', 'SubjectEthnicity',
       'SubjectRace', 'SubjectResistance', 'SubjectSex', 'TimeOccurred',
       'WeaponUsed1', 'WeaponUsed2', 'WeaponUsed4', 'WeaponUsed5', 'XCoord',
       'YCoord', 'ClearanceDate', 'ClearanceStatus', 'CouncilDistrict',
       'GOCensusTract', 'GODistrict', 'GOHighestOffenseDesc', 'GOLocation',
       'GOLocationZip', 'GOReportDate', 'NIBRS', 'MasterSubjectID',
       'OfficerYrsServ', 'WeaponUsed3', 'UF'],
      dtype='object')


In [26]:
APDdis=gpd.read_file("APD Districts.geojson")
AUF2015.head(5)

APDdis.head(2)
APDdis.district_name.unique()
#SMC= gpd.read_file("Single Member Council Districts.geojson")
SMC.plot()
#APDdis

array(['STS', 'C3 UT-2', 'ADAM 1', 'FRANK 3', 'DAVID 2', 'DAVID 3',
       'GEORGE 2', 'B4 UT', 'C3 UT-3', 'HENRY 1', 'GEORGE 4', 'IDA 2',
       'HENRY 4', 'HENRY 3', 'GEORGE 5', 'ADAM 4', 'BAKER 4', 'PUBLIC',
       'LND', 'A1 UT', 'CHARLIE 3', 'BAKER 3', 'BAKER PARK PD', 'FRANK 2',
       'DAVID 4', 'EDWARD 3', 'IDA 1', 'STP', 'HENRY 2', 'MTP', 'AOA',
       'FRANK 1', 'DAVID 1', 'GEORGE 1', 'CHARLIE 4', 'CHARLIE 1',
       'BAKER 1', 'FRANK 4', 'CHARLIE 2', 'ADAM 3', 'BAKER 2', 'GEORGE 3',
       'EDWARD 1', 'EDWARD 4', 'MTS', 'IDA 3', 'B1 UT-2', 'ADAM 2',
       'EDWARD 2', 'B1 UT-1', 'IDA 4', 'DAVID PARK PD', 'C3 UT-1'], dtype=object)

In [70]:
#Removing white space from AUF2015 and AAnn keys. 

AUF2015.columns = AUF2015.columns.str.replace('\s+','')
print(AUF2015.keys())
AAnn.columns = AAnn.columns.str.replace('\s+','')
print(AAnn.keys())

Index(['RIN', 'PrimaryKey', 'DateOccurred', 'TimeOccurred', 'Location',
       'AreaCommand', 'NatureofContact', 'ReasonDesc', 'R2RLevel',
       'MasterSubjectID', 'SubjectSex', 'SubjectRace', 'SubjectEthnicity',
       'SubjectConductDesc', 'SubjectResistance', 'WeaponUsed1', 'WeaponUsed2',
       'WeaponUsed3', 'WeaponUsed4', 'WeaponUsed5', 'NumberShots',
       'SubjectEffects', 'EffectonOfficer', 'OfficerOrganizationDesc',
       'OfficerCommissionDate', 'OfficerYrsofService', 'X-Coordinate',
       'Y-Coordinate', 'CouncilDistrict'],
      dtype='object')
Index(['GOPrimaryKey', 'CouncilDistrict', 'GOHighestOffenseDesc',
       'HighestNIBRS/UCROffenseDescription', 'GOReportDate', 'GOLocation',
       'ClearanceStatus', 'ClearanceDate', 'GODistrict', 'GOLocationZip',
       'GOCensusTract', 'GOXCoordinate', 'GOYCoordinate'],
      dtype='object')


In [163]:
#Creating dictionary of common keys (incident identifiers) between datasets
AAnn.keys()

DupKey=set(AAnn['GOPrimaryKey']) & set(AUF2015['Key'])
KeyDict = dict.fromkeys(DupKey)
#print(KeyDict)
dupkey = KeyDict.keys()
dupkeylist = list(dupkey)
print(dupkeylist)

[20152811520, 20152650246, 20152220678, 20152860682, 20152071178, 20151991821, 20151411727, 20151260176, 2015490066, 20153001492, 20151370776, 20152261144, 2015190045, 20151790622, 20151241249, 20152630305, 20153441827, 2015730212, 20152350249, 2015390765, 2015351350, 20153251387, 20153501247, 2015360578, 2015641667, 20152630338, 20152090184, 20151832142, 2015110225, 20151361107, 20151161429, 2015872086, 20151860826, 20152240220, 20151320161, 20152830053, 20151940198, 20152390249, 20151221865, 2015520363, 20152391788, 20153150061, 20151500403, 20151420020, 2015510647, 2015100025, 2015431291, 2015141501, 20153221764, 2015960708, 20152280714, 201580171, 20151390349, 20153121425, 20151590554, 20153290912, 20152910497, 20153620645, 20153371304, 20152930473, 20152940202, 20151830188, 20151370416, 20151731897, 20152471738, 20152320190, 20153450174, 20151790272, 20151371969, 20152450248, 20153621197, 20151290063, 20151492303, 20151000276, 20151051991, 20151721693, 20151141090, 20152471269, 20

In [ ]:
#Just some info for us:

print('There are', len(set(AAnn['GOPrimaryKey'])), 'incidents in AAnn 2015 annual crime')
print('There are', len(set(AUF2015['PrimaryKey'])), 'incidents in AUF2015 use of force')
print('There are', len(KeyDict),'common incidents between AAnn and AUF2015')
PercentCommonFromUF= len(KeyDict)/(len(set(AUF2015['PrimaryKey'])))
print('This means', PercentCommonFromUF, '% of UF incidents are recorded in AAnn annual crime dataset.')

    

40062
3302
38573
192


41683

In [92]:
#Finding NaNs in datasets

#How many NaNs are in each column?
AAnn.isnull().sum()
AUF2015.isnull().sum()

#Create table of rows with NaNs
AAnn_nan_rows = AAnn[AAnn.isnull().T.any().T]
AUF2015_nan_rows = AUF2015[AUF2015.isnull().T.any().T]

#Where specifically are the NaNs? Create table of Boolean true/false. True = NaN
#False = not a NaN
AnnNaNLoc= AAnn.isnull()
AUF2015NanLoc= AUF2015.isnull()


In [2]:
#AAnn
AAnn_vars = ['GOPrimaryKey', 'GOHighestOffenseDesc', 'GO', 'GOReportDate', 
             'GOLocation','GOLocationZip', 'GOCensusTract', 'GOXCoordinate', 'GOYCoordinate']

#AUF2015
AUF2015_vars = ['PrimaryKey', 'DateOccurred', 'TimeOccurred', 'Location', 'NatureOfContact', 
                'SubjectEffects', 'OfficerOrganizationDesc', 'OfficerCommissionDate', 'OfficerYrsofService', 
                'X-Coordinate', 'Y-Coordinate'] 

#Printing 

#To find out if the columns we care about have NaNs in them:

#AAnn
AAnn_nan_rows_GOPrimaryKey = AAnn[AAnn['GOPrimaryKey'].isnull()]
AAnn_nan_rows_GOHighestOffenseDesc = AAnn[AAnn['GOHighestOffenseDesc'].isnull()]
AAnn_nan_rows_HighNIBRS = AAnn[AAnn['HighestNIBRS/UCROffenseDescription'].isnull()]
AAnn_nan_rows_GOReportDate = AAnn[AAnn['GOReportDate'].isnull()]
AAnn_nan_rows_GOLocation = AAnn[AAnn['GOLocation'].isnull()]
AAnn_nan_rows_GOLocationZip = AAnn[AAnn['GOLocationZip'].isnull()]
AAnn_nan_rows_GOCensusTract = AAnn[AAnn['GOCensusTract'].isnull()]
AAnn_nan_rows_GOXCoordinate = AAnn[AAnn['GOXCoordinate'].isnull()]
AAnn_nan_rows_GOYCoordinate = AAnn[AAnn['GOYCoordinate'].isnull()]

#AUF2015
AUF2015_nan_rows_PrimaryKey = AUF2015[AUF2015['PrimaryKey'].isnull()]
AUF2015_nan_rows_DateOccurred = AUF2015[AUF2015['DateOccurred'].isnull()]
AUF2015_nan_rows_TimeOccurred = AUF2015[AUF2015['TimeOccurred'].isnull()]
AUF2015_nan_rows_Location = AUF2015[AUF2015['Location'].isnull()]
AUF2015_nan_rows_NatureofContact = AUF2015[AUF2015['NatureofContact'].isnull()]
AUF2015_nan_rows_SubjectEffects = AUF2015[AUF2015['SubjectEffects'].isnull()]
AUF2015_nan_rows_OfficerOrganizationDesc = AUF2015[AUF2015['OfficerOrganizationDesc'].isnull()]
AUF2015_nan_rows_OfficerCommissionDate = AUF2015[AUF2015['OfficerCommissionDate'].isnull()]
AUF2015_nan_rows_OfficerYrsofService = AUF2015[AUF2015['OfficerYrsofService'].isnull()]
AUF2015_nan_rows_XCoordinate = AUF2015[AUF2015['X-Coordinate'].isnull()]
AUF2015_nan_rows_YCoordinate = AUF2015[AUF2015['Y-Coordinate'].isnull()]

KeyError: 'GOPrimaryKey'

In [ ]:
stack_nan = stack[stack.isnull().T.any().T]

In [200]:
stack.keys()

Index(['Key', 'CouncilDistrict', 'UF', 'XCoord', 'YCoord', 'RIN',
       'DateOccurred', 'TimeOccurred', 'ClearanceDate', 'GOReportDate',
       'R2RLevel', 'NIBRS', 'AreaCommand', 'Location', 'GOCensusTract',
       'GODistrict', 'GOLocation', 'GOLocationZip', 'OfficerEffects',
       'OfficerCommissionDate', 'OfficerYrsServ', 'OfficerOrganizationDesc',
       'ReasonDesc', 'SubjectConductDesc', 'SubjectEffects',
       'SubjectEthnicity', 'SubjectRace', 'SubjectResistance', 'SubjectSex',
       'NatureOfContact', 'GOHighestOffenseDesc', 'NumberShots', 'WeaponUsed1',
       'WeaponUsed2', 'WeaponUsed3', 'WeaponUsed4', 'WeaponUsed5',
       'ClearanceStatus', 'MasterSubjectID'],
      dtype='object')

In [219]:
#stack_nan = stack[['Key', 'UF', 'GOHighestOffenseDesc', 'NIBRS', 'GOReportDate', 'GOLocation', 'ClearanceStatus', 'ClearanceDate', 'GODistrict', 'GOLocationZip', 'GOCensusTract']]

#nan= stack_nan[stack_nan.isnull().T.any().T]
#print(nan.iloc[3000][0])
#nan

#stack_nan

for row in stack:
    if stack.UF == False:
        print(stack[['Key', 'UF', 'GOHighestOffenseDesc', 'NIBRS', 'GOReportDate', 'GOLocation', 'ClearanceStatus', 'ClearanceDate', 'GODistrict', 'GOLocationZip', 'GOCensusTract']])


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [226]:
print(AAnn.keys())
AUF2015.keys()

Index(['GOPrimaryKey', 'Council_District', 'GOHighestOffenseDesc', 'NIBRS',
       'GOReportDate', 'GOLocation', 'ClearanceStatus', 'ClearanceDate',
       'GODistrict', 'GOLocationZip', 'GOCensusTract', 'GOXCoordinate',
       'GOYCoordinate'],
      dtype='object')


Index(['RIN', 'Key', 'DateOccurred', 'TimeOccurred', 'Location', 'AreaCommand',
       'NatureofContact', 'ReasonDesc', 'R2RLevel', 'MasterSubjectID',
       'SubjectSex', 'SubjectRace', 'SubjectEthnicity', 'SubjectConductDesc',
       'SubjectResistance', 'WeaponUsed1', 'WeaponUsed2', 'WeaponUsed3',
       'WeaponUsed4', 'WeaponUsed5', 'NumberShots', 'SubjectEffects',
       'OfficerEffects', 'OfficerOrganizationDesc', 'OfficerCommissionDate',
       'OfficerYrsServ', 'X-Coordinate', 'Y-Coordinate', 'CouncilDistrict'],
      dtype='object')

In [224]:
null = stack.loc[(stack['UF'] == False)] 
          #& df['other_column'].isin(some_values)]
null.isnull().sum()

Key                           0
CouncilDistrict               0
UF                            0
XCoord                        0
YCoord                        0
RIN                           0
DateOccurred                  0
TimeOccurred                  0
ClearanceDate                44
GOReportDate                  0
R2RLevel                      1
NIBRS                         0
AreaCommand                   0
Location                      0
GOCensusTract                 1
GODistrict                    0
GOLocation                   17
GOLocationZip                 1
OfficerEffects              112
OfficerCommissionDate         2
OfficerYrsServ                0
OfficerOrganizationDesc       2
ReasonDesc                    1
SubjectConductDesc          360
SubjectEffects                9
SubjectEthnicity             88
SubjectRace                  76
SubjectResistance             1
SubjectSex                   76
NatureOfContact            1673
GOHighestOffenseDesc          0
NumberSh

In [3]:
AAnn.isnull().sum()

GO Primary Key                              0
Council District                          172
GO Highest Offense Desc                     0
Highest NIBRS/UCR Offense Description       0
GO Report Date                              0
GO Location                               481
Clearance Status                         1924
Clearance Date                           1924
GO District                                 0
GO Location Zip                           159
GO Census Tract                           159
GO X Coordinate                           592
GO Y Coordinate                           592
dtype: int64